# Produce press release results from the new pipeline, predstore

In [47]:
import requests
import pandas as pd
import json
import os
from pathlib import Path
import numpy as np

#import matplotlib.pyplot as plt

# External
import geopandas as gpd
import sqlalchemy as sa

# VIEWS
from viewser.operations import fetch
from views_mapper2.mapper2 import *
from views_mapper2.BBoxWriter import *
from views_mapper2.dictionary_writer import *
from ingester3.config import source_db_path
from views_forecasts.extensions import *
from viewser import Queryset, Column
from views_mapper2.mapper2 import Mapper2

## General settings

### Set end of history

In [48]:
eoh = 548
EndOfHistory = 548

## Fetch old pipeline forecasts

In [49]:
eoh = 548

old_run_id = 46
old_run_name_Aug = f'cm_genetic_ensemble_f{eoh}'
ViewsMetadata().with_name(name=old_run_name_Aug).fetch()
old_data_Aug = pd.DataFrame.forecasts.read_store(run=old_run_id, name=old_run_name_Aug)
old_data_Aug['old_main_mean_Aug'] = np.expm1(old_data_Aug['step_combined'])
old_data_Aug


pr_46_cm_genetic_ensemble_f548.parquet


feature              step_combined  old_main_mean_Aug
month_id country_id                                  
549      1                0.004368           0.004377
         2                0.002407           0.002410
         3                0.040729           0.041569
         4                0.063545           0.065607
         5                0.007164           0.007189
...                            ...                ...
584      242              0.315599           0.371080
         243              0.641476           0.899281
         244              0.286496           0.331753
         245              5.589598         266.627887
         246              2.507721          11.276918

[6876 rows x 2 columns]

In [50]:
eoh = 549

old_run_id = 46
old_run_name_Sep = f'cm_genetic_ensemble_f{eoh}'
ViewsMetadata().with_name(name=old_run_name_Sep).fetch()
old_data_Sep = pd.DataFrame.forecasts.read_store(run=old_run_id, name=old_run_name_Sep)
old_data_Sep['old_main_mean_Sep'] = np.expm1(old_data_Sep['step_combined'])
old_data_Sep


pr_46_cm_genetic_ensemble_f549.parquet


feature              step_combined  old_main_mean_Sep
month_id country_id                                  
550      1                0.001749           0.001751
         2                0.003540           0.003547
         3                0.043151           0.044096
         4                0.742192           1.100534
         5                0.009478           0.009523
...                            ...                ...
585      242              0.343062           0.409256
         243              0.629551           0.876767
         244              0.317161           0.373224
         245              5.258471         191.187366
         246              3.352334          27.569344

[6876 rows x 2 columns]

In [51]:
eoh = 550

old_run_id = 46
old_run_name_Oct = f'cm_genetic_ensemble_f{eoh}'
ViewsMetadata().with_name(name=old_run_name_Oct).fetch()
old_data_Oct = pd.DataFrame.forecasts.read_store(run=old_run_id, name=old_run_name_Oct)
old_data_Oct['old_main_mean_Oct'] = np.expm1(old_data_Oct['step_combined'])
old_data_Oct

pr_46_cm_genetic_ensemble_f550.parquet


feature              step_combined  old_main_mean_Oct
month_id country_id                                  
551      1                0.014763           0.014873
         2                0.003730           0.003736
         3                0.003946           0.003954
         4                1.719697           4.582836
         5                0.009505           0.009550
...                            ...                ...
586      242              0.335130           0.398122
         243              0.876515           1.402512
         244              0.321653           0.379406
         245              5.512356         246.733989
         246              3.169583          22.797564

[6876 rows x 2 columns]

In [52]:
## Merge old and new df

all_old_forecasts = pd.merge(old_data_Aug, old_data_Sep, on=["country_id", "month_id"], how="left").reset_index().set_index(['country_id', 'month_id'])
all_old_forecasts = pd.merge(all_old_forecasts, old_data_Oct, on=["country_id", "month_id"], how="left").reset_index().set_index(['country_id', 'month_id'])
all_old_forecasts 


,feature,step_combined_x,old_main_mean_Aug,step_combined_y,old_main_mean_Sep,step_combined,old_main_mean_Oct
country_id,month_id,,,,,,
1,549,0.004368,0.004377,NaN,NaN,NaN,NaN
2,549,0.002407,0.002410,NaN,NaN,NaN,NaN
3,549,0.040729,0.041569,NaN,NaN,NaN,NaN
4,549,0.063545,0.065607,NaN,NaN,NaN,NaN
5,549,0.007164,0.007189,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
242,584,0.315599,0.371080,0.388488,0.474749,0.369962,0.447680
243,584,0.641476,0.899281,0.500686,0.649852,0.734187,1.083787
244,584,0.286496,0.331753,0.398782,0.490009,0.281078,0.324557


In [53]:
cols_to_keep = ['old_main_mean_Aug', 'old_main_mean_Sep', 'old_main_mean_Oct']

# creating new df
filtered_all_old_forecasts = all_old_forecasts[cols_to_keep]
filtered_all_old_forecasts

,feature,old_main_mean_Aug,old_main_mean_Sep,old_main_mean_Oct
country_id,month_id,,,
1,549,0.004377,NaN,NaN
2,549,0.002410,NaN,NaN
3,549,0.041569,NaN,NaN
4,549,0.065607,NaN,NaN
5,549,0.007189,NaN,NaN
...,...,...,...,...
242,584,0.371080,0.474749,0.447680
243,584,0.899281,0.649852,1.083787
244,584,0.331753,0.490009,0.324557


## Fetch identifiers

In [54]:
# Identifiers

identifiers_cm = (Queryset("identifiers", "country_month")

.with_column(Column('year', from_loa = "country_year", from_column = 'year_id'))
.with_column(Column('month', from_loa = "month", from_column = 'month'))
.with_column(Column('isoab', from_loa = "country", from_column = 'isoab'))
.with_column(Column('country', from_loa = "country", from_column = 'name'))

)
      
identifiers_cm = identifiers_cm.publish().fetch()

print(f"A dataset with {len(identifiers_cm.columns)} columns, with "
      f"data between t {min(identifiers_cm.index.get_level_values(0))} "
      f"and {max(identifiers_cm.index.get_level_values(0))}. "
      f"({len(np.unique(identifiers_cm.index.get_level_values(1)))} units)"
     )

Queryset identifiers read successfully 
A dataset with 4 columns, with data between t 1 and 852. (213 units)


In [55]:
identifiers_cm

year  month isoab              country
month_id country_id                                        
1        1           1980      1   GUY               Guyana
         2           1980      1   SUR             Suriname
         3           1980      1   TTO  Trinidad and Tobago
         4           1980      1   VEN            Venezuela
         5           1980      1   WSM                Samoa
...                   ...    ...   ...                  ...
852      242         2050     12   TZA             Tanzania
         243         2050     12   MAR              Morocco
         244         2050     12   MRT           Mauritania
         245         2050     12   SDN                Sudan
         246         2050     12   SSD          South Sudan

[158230 rows x 4 columns]

In [56]:
## Merge

final_old_forecasts = pd.merge(filtered_all_old_forecasts, identifiers_cm, on=["country_id", "month_id"], how="left")

final_old_forecasts

,,old_main_mean_Aug,old_main_mean_Sep,old_main_mean_Oct,year,month,isoab,country
country_id,month_id,,,,,,,
1,549,0.004377,NaN,NaN,2025,9,GUY,Guyana
2,549,0.002410,NaN,NaN,2025,9,SUR,Suriname
3,549,0.041569,NaN,NaN,2025,9,TTO,Trinidad and Tobago
4,549,0.065607,NaN,NaN,2025,9,VEN,Venezuela
5,549,0.007189,NaN,NaN,2025,9,WSM,Samoa
...,...,...,...,...,...,...,...,...
242,584,0.371080,0.474749,0.447680,2028,8,TZA,Tanzania
243,584,0.899281,0.649852,1.083787,2028,8,MAR,Morocco
244,584,0.331753,0.490009,0.324557,2028,8,MRT,Mauritania


In [57]:
filter_old_forecasts_by_cy = final_old_forecasts[final_old_forecasts['year'] == 2026]
filter_old_forecasts_by_cy

,,old_main_mean_Aug,old_main_mean_Sep,old_main_mean_Oct,year,month,isoab,country
country_id,month_id,,,,,,,
1,553,0.010249,0.006543,0.020103,2026,1,GUY,Guyana
2,553,0.003276,0.009719,0.003741,2026,1,SUR,Suriname
3,553,0.065731,0.013692,0.014244,2026,1,TTO,Trinidad and Tobago
4,553,0.190127,1.196772,4.553550,2026,1,VEN,Venezuela
5,553,0.002279,0.003914,0.004398,2026,1,WSM,Samoa
...,...,...,...,...,...,...,...,...
242,564,0.211946,0.155744,0.176861,2026,12,TZA,Tanzania
243,564,0.666504,0.473180,1.082578,2026,12,MAR,Morocco
244,564,0.254317,0.210112,0.233726,2026,12,MRT,Mauritania


In [58]:
sorted_filtered_forecasts_by_cy = (
    filter_old_forecasts_by_cy
    .groupby(['country_id', 'country', 'year'], as_index=False)
    .agg({
        'old_main_mean_Aug': 'sum',
        'old_main_mean_Sep': 'sum',
        'old_main_mean_Oct': 'sum'
    })
    .sort_values(by='old_main_mean_Oct', ascending=False)
)

sorted_filtered_forecasts_by_cy.head(15)

,country,year,old_main_mean_Aug,old_main_mean_Sep,old_main_mean_Oct
103,Ukraine,2026,9203.366798,9690.661848,10916.049336
189,Sudan,2026,3119.735102,2376.036815,6008.272803
122,Pakistan,2026,3387.799055,4322.146210,5371.664358
176,Israel,2026,9205.149142,5688.235877,4206.485838
69,Nigeria,2026,3352.680643,3341.439997,3454.894721
106,Somalia,2026,4720.756764,3498.960159,2993.858197
53,Ethiopia,2026,1015.931909,1860.981160,2250.318184
44,Burkina Faso,2026,1755.066051,1518.343086,1640.325846
134,Myanmar,2026,1631.379325,1415.472580,1508.643220
110,Yemen,2026,1568.583563,2191.837899,1492.575145


## Fetch new pipeline forecasts

In [11]:

new_runs_cm = f'pink_ponyclub_predictions_forecasting'
ViewsMetadata().with_name(name=new_runs_cm).fetch()

,id,name,file_name,runs_id,model_generations_id,user_name,spatial_loa,temporal_loa,ds,osa,time_min,time_max,space_min,space_max,steps,target,prediction_columns,date_written,description,deleted
0,23157,pink_ponyclub_predictions_forecasting_20251009...,pr_67_pink_ponyclub_predictions_forecasting_20...,67,1,‘xiaolong’,c,m,False,False,550,585,1,246,[],pred_ln_ged_sb_dep,[],2025-10-09 11:06:04.322679+00:00,None,False
1,23192,pink_ponyclub_predictions_forecasting_20251016...,pr_67_pink_ponyclub_predictions_forecasting_20...,67,1,jim,c,m,False,False,549,584,1,246,[],pred_ln_ged_sb_dep,[],2025-10-16 13:32:41.156497+00:00,None,False
2,23256,pink_ponyclub_predictions_forecasting_20251024...,pr_67_pink_ponyclub_predictions_forecasting_20...,67,1,‘xiaolong’,c,m,False,False,550,585,1,246,[],pred_ln_ged_sb_dep,[],2025-10-24 14:54:22.122957+00:00,None,False
3,23333,pink_ponyclub_predictions_forecasting_20251027...,pr_67_pink_ponyclub_predictions_forecasting_20...,67,1,‘xiaolong’,c,m,False,False,550,585,1,246,[],pred_ln_ged_sb_dep,[],2025-10-27 11:39:07.875302+00:00,None,False
4,23502,pink_ponyclub_predictions_forecasting_20251126...,pr_68_pink_ponyclub_predictions_forecasting_20...,68,1,‘xiaolong’,c,m,False,False,551,586,1,246,[],pred_lr_ged_sb,[],2025-11-26 13:07:39.539427+00:00,None,False
5,23608,pink_ponyclub_predictions_forecasting_20251127...,pr_68_pink_ponyclub_predictions_forecasting_20...,68,1,‘xiaolong’,c,m,False,False,551,586,1,246,[],pred_lr_ged_sb,[],2025-11-27 13:23:31.137537+00:00,None,False


In [12]:
new_run_id = 67
# Select the last run in the df - this is the most recent, and presumably therefore the one we want
new_data_name_Sep = ViewsMetadata().with_name(name=new_runs_cm).fetch().iloc[-3]['name']
new_data_name_Sep

'pink_ponyclub_predictions_forecasting_20251027_113907'

In [13]:
new_data_Sep = pd.DataFrame.forecasts.read_store(run=new_run_id, name=new_data_name_Sep)

new_data_Sep['new_main_mean_Sep'] = np.expm1(new_data_Sep['pred_ln_ged_sb_dep'])
new_data_Sep

pr_67_pink_ponyclub_predictions_forecasting_20251027_113907.parquet


pred_ln_ged_sb_dep  new_main_mean_Sep
month_id country_id                                       
550      1                     0.003773           0.003780
         2                     0.003762           0.003769
         3                     0.011403           0.011468
         4                     0.713244           1.040601
         5                     0.003762           0.003769
...                                 ...                ...
585      242                   0.105974           0.111793
         243                   0.378329           0.459844
         244                   0.107760           0.113780
         245                   3.757777          41.853047
         246                   2.260278           8.585755

[6876 rows x 2 columns]

In [ ]:
new_run_id = 68
# Select the last run in the df - this is the most recent, and presumably therefore the one we want
new_data_name_Oct = ViewsMetadata().with_name(name=new_runs_cm).fetch().iloc[-1]['name']

new_data_name_Oct

'pink_ponyclub_predictions_forecasting_20251127_124749'

In [25]:
new_data_Oct = pd.DataFrame.forecasts.read_store(run=new_run_id, name=new_data_name_Oct)
new_data_Oct

pr_68_pink_ponyclub_predictions_forecasting_20251127_124749.parquet


pred_lr_ged_sb
month_id country_id                
551      1            [0.062928505]
         2           [0.0040881606]
         3           [0.0074733794]
         4               [4.773798]
         5           [0.0040881606]
...                             ...
586      242           [0.16878237]
         243            [0.5493169]
         244           [0.13097902]
         245             [66.52192]
         246              [12.6189]

[6876 rows x 1 columns]

In [26]:
new_data_Oct["new_main_mean_Oct"] = new_data_Oct["pred_lr_ged_sb"].str[0]
new_data_Oct

pred_lr_ged_sb  new_main_mean_Oct
month_id country_id                                   
551      1            [0.062928505]           0.062929
         2           [0.0040881606]           0.004088
         3           [0.0074733794]           0.007473
         4               [4.773798]           4.773798
         5           [0.0040881606]           0.004088
...                             ...                ...
586      242           [0.16878237]           0.168782
         243            [0.5493169]           0.549317
         244           [0.13097902]           0.130979
         245             [66.52192]          66.521919
         246              [12.6189]          12.618900

[6876 rows x 2 columns]

In [27]:
all_new_forecasts = pd.merge(new_data_Sep, new_data_Oct, on=["country_id", "month_id"], how="left").reset_index().set_index(['country_id', 'month_id'])
all_new_forecasts

,,pred_ln_ged_sb_dep,new_main_mean_Sep,pred_lr_ged_sb,new_main_mean_Oct
country_id,month_id,,,,
1,550,0.003773,0.003780,NaN,NaN
2,550,0.003762,0.003769,NaN,NaN
3,550,0.011403,0.011468,NaN,NaN
4,550,0.713244,1.040601,NaN,NaN
5,550,0.003762,0.003769,NaN,NaN
...,...,...,...,...,...
242,585,0.105974,0.111793,[0.1330415],0.133042
243,585,0.378329,0.459844,[0.5617649],0.561765
244,585,0.107760,0.113780,[0.11769368],0.117694


In [28]:
cols_to_keep = ['new_main_mean_Sep', 'new_main_mean_Oct']

# creating new df
filtered_all_new_forecasts = all_new_forecasts[cols_to_keep]
filtered_all_new_forecasts

,,new_main_mean_Sep,new_main_mean_Oct
country_id,month_id,,
1,550,0.003780,NaN
2,550,0.003769,NaN
3,550,0.011468,NaN
4,550,1.040601,NaN
5,550,0.003769,NaN
...,...,...,...
242,585,0.111793,0.133042
243,585,0.459844,0.561765
244,585,0.113780,0.117694


In [32]:
## Merge

filtered_all_new_forecasts = pd.merge(filtered_all_new_forecasts, identifiers_cm, on=["country_id", "month_id"], how="left")

filtered_all_new_forecasts

,,new_main_mean_Sep,new_main_mean_Oct,year,month,isoab,country
country_id,month_id,,,,,,
1,550,0.003780,NaN,2025,10,GUY,Guyana
2,550,0.003769,NaN,2025,10,SUR,Suriname
3,550,0.011468,NaN,2025,10,TTO,Trinidad and Tobago
4,550,1.040601,NaN,2025,10,VEN,Venezuela
5,550,0.003769,NaN,2025,10,WSM,Samoa
...,...,...,...,...,...,...,...
242,585,0.111793,0.133042,2028,9,TZA,Tanzania
243,585,0.459844,0.561765,2028,9,MAR,Morocco
244,585,0.113780,0.117694,2028,9,MRT,Mauritania


In [33]:
filter_new_forecasts_by_cy = filtered_all_new_forecasts[filtered_all_new_forecasts['year'] == 2026]
filter_new_forecasts_by_cy  

,,new_main_mean_Sep,new_main_mean_Oct,year,month,isoab,country
country_id,month_id,,,,,,
1,553,0.008633,0.023691,2026,1,GUY,Guyana
2,553,0.008499,0.009745,2026,1,SUR,Suriname
3,553,0.026636,0.012815,2026,1,TTO,Trinidad and Tobago
4,553,0.893620,4.784820,2026,1,VEN,Venezuela
5,553,0.008464,0.010034,2026,1,WSM,Samoa
...,...,...,...,...,...,...,...
242,564,0.048130,0.068764,2026,12,TZA,Tanzania
243,564,0.258691,0.330855,2026,12,MAR,Morocco
244,564,0.092271,0.080384,2026,12,MRT,Mauritania


In [37]:
sorted_filtered_new_forecasts_by_cy = (
    filter_new_forecasts_by_cy
    .groupby(['country_id', 'country', 'year'], as_index=False)
    .agg({
        'new_main_mean_Sep': 'sum',
        'new_main_mean_Oct': 'sum'
    })
    .sort_values(by='new_main_mean_Oct', ascending=False)
)

sorted_filtered_new_forecasts_by_cy.head(10)

,country,year,new_main_mean_Sep,new_main_mean_Oct
103,Ukraine,2026,20983.443134,28317.156250
176,Israel,2026,7120.487814,7710.166016
189,Sudan,2026,2014.057250,4262.010742
122,Pakistan,2026,1877.161177,1986.209839
69,Nigeria,2026,1634.492180,1891.390625
53,Ethiopia,2026,1489.944279,1835.902832
106,Somalia,2026,1764.219552,1692.250488
177,Syria,2026,411.630041,1447.200684
110,Yemen,2026,894.024153,1270.351440
44,Burkina Faso,2026,1175.438937,1228.034790


## Just Sept

In [38]:
new_data_Sep

pred_ln_ged_sb_dep  new_main_mean_Sep
month_id country_id                                       
550      1                     0.003773           0.003780
         2                     0.003762           0.003769
         3                     0.011403           0.011468
         4                     0.713244           1.040601
         5                     0.003762           0.003769
...                                 ...                ...
585      242                   0.105974           0.111793
         243                   0.378329           0.459844
         244                   0.107760           0.113780
         245                   3.757777          41.853047
         246                   2.260278           8.585755

[6876 rows x 2 columns]

In [39]:
## Merge

final_sep_forecasts = pd.merge(new_data_Sep, identifiers_cm, on=["country_id", "month_id"], how="left")

final_sep_forecasts

,,pred_ln_ged_sb_dep,new_main_mean_Sep,year,month,isoab,country
country_id,month_id,,,,,,
1,550,0.003773,0.003780,2025,10,GUY,Guyana
2,550,0.003762,0.003769,2025,10,SUR,Suriname
3,550,0.011403,0.011468,2025,10,TTO,Trinidad and Tobago
4,550,0.713244,1.040601,2025,10,VEN,Venezuela
5,550,0.003762,0.003769,2025,10,WSM,Samoa
...,...,...,...,...,...,...,...
242,585,0.105974,0.111793,2028,9,TZA,Tanzania
243,585,0.378329,0.459844,2028,9,MAR,Morocco
244,585,0.107760,0.113780,2028,9,MRT,Mauritania


In [40]:
final_sep_forecasts_by_cy = final_sep_forecasts[final_sep_forecasts['year'] == 2026]
final_sep_forecasts_by_cy  

,,pred_ln_ged_sb_dep,new_main_mean_Sep,year,month,isoab,country
country_id,month_id,,,,,,
1,553,0.008596,0.008633,2026,1,GUY,Guyana
2,553,0.008463,0.008499,2026,1,SUR,Suriname
3,553,0.026287,0.026636,2026,1,TTO,Trinidad and Tobago
4,553,0.638490,0.893620,2026,1,VEN,Venezuela
5,553,0.008428,0.008464,2026,1,WSM,Samoa
...,...,...,...,...,...,...,...
242,564,0.047008,0.048130,2026,12,TZA,Tanzania
243,564,0.230072,0.258691,2026,12,MAR,Morocco
244,564,0.088259,0.092271,2026,12,MRT,Mauritania


In [42]:
sorted_filtered_sep_forecasts_by_cy = (
    final_sep_forecasts_by_cy
    .groupby(['country_id', 'country', 'year'], as_index=False)
    .agg({
        'new_main_mean_Sep': 'sum',
    })
    .sort_values(by='new_main_mean_Sep', ascending=False)
)

sorted_filtered_sep_forecasts_by_cy.head(10)

,country,year,new_main_mean_Sep
103,Ukraine,2026,20983.443134
176,Israel,2026,7120.487814
189,Sudan,2026,2014.057250
122,Pakistan,2026,1877.161177
106,Somalia,2026,1764.219552
69,Nigeria,2026,1634.492180
53,Ethiopia,2026,1489.944279
44,Burkina Faso,2026,1175.438937
134,Myanmar,2026,1129.306468
47,Mali,2026,932.260525


## Just Oct

In [43]:
new_data_Oct

pred_lr_ged_sb  new_main_mean_Oct
month_id country_id                                   
551      1            [0.062928505]           0.062929
         2           [0.0040881606]           0.004088
         3           [0.0074733794]           0.007473
         4               [4.773798]           4.773798
         5           [0.0040881606]           0.004088
...                             ...                ...
586      242           [0.16878237]           0.168782
         243            [0.5493169]           0.549317
         244           [0.13097902]           0.130979
         245             [66.52192]          66.521919
         246              [12.6189]          12.618900

[6876 rows x 2 columns]

In [44]:
## Merge

final_oct_forecasts = pd.merge(new_data_Oct, identifiers_cm, on=["country_id", "month_id"], how="left")

final_oct_forecasts

,,pred_lr_ged_sb,new_main_mean_Oct,year,month,isoab,country
country_id,month_id,,,,,,
1,551,[0.062928505],0.062929,2025,11,GUY,Guyana
2,551,[0.0040881606],0.004088,2025,11,SUR,Suriname
3,551,[0.0074733794],0.007473,2025,11,TTO,Trinidad and Tobago
4,551,[4.773798],4.773798,2025,11,VEN,Venezuela
5,551,[0.0040881606],0.004088,2025,11,WSM,Samoa
...,...,...,...,...,...,...,...
242,586,[0.16878237],0.168782,2028,10,TZA,Tanzania
243,586,[0.5493169],0.549317,2028,10,MAR,Morocco
244,586,[0.13097902],0.130979,2028,10,MRT,Mauritania


In [45]:
final_oct_forecasts_by_cy = final_oct_forecasts[final_oct_forecasts['year'] == 2026]
final_oct_forecasts_by_cy  

,,pred_lr_ged_sb,new_main_mean_Oct,year,month,isoab,country
country_id,month_id,,,,,,
1,553,[0.023691183],0.023691,2026,1,GUY,Guyana
2,553,[0.00974494],0.009745,2026,1,SUR,Suriname
3,553,[0.012815234],0.012815,2026,1,TTO,Trinidad and Tobago
4,553,[4.7848196],4.784820,2026,1,VEN,Venezuela
5,553,[0.010034102],0.010034,2026,1,WSM,Samoa
...,...,...,...,...,...,...,...
242,564,[0.06876383],0.068764,2026,12,TZA,Tanzania
243,564,[0.33085474],0.330855,2026,12,MAR,Morocco
244,564,[0.08038359],0.080384,2026,12,MRT,Mauritania


In [46]:
sorted_filtered_oct_forecasts_by_cy = (
    final_oct_forecasts_by_cy
    .groupby(['country_id', 'country', 'year'], as_index=False)
    .agg({
        'new_main_mean_Oct': 'sum',
    })
    .sort_values(by='new_main_mean_Oct', ascending=False)
)

sorted_filtered_oct_forecasts_by_cy.head(10)

,country,year,new_main_mean_Oct
103,Ukraine,2026,28317.156250
176,Israel,2026,7710.166016
189,Sudan,2026,4262.010742
122,Pakistan,2026,1986.209839
69,Nigeria,2026,1891.390625
53,Ethiopia,2026,1835.902832
106,Somalia,2026,1692.250488
177,Syria,2026,1447.200684
110,Yemen,2026,1270.351440
44,Burkina Faso,2026,1228.034790
